In [2]:
#importing useful packages
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.io import ascii
import astropy.time
from astropy.time import Time
import astropy.table
from astropy.table import Table, Column, MaskedColumn
from astroplan import Observer
from astroquery.jplhorizons import Horizons
import pandas as pd
import csv
import astropy.time
import dateutil.parser

### Reading CSV file of dates targets were observed
raw_target_info = pd.read_csv("input_obs_data.csv")

In [3]:
### Creating new dataframe with first date each target was observed
subset_df = raw_target_info[["Target","Date"]]
grouped_and_first_df = subset_df.groupby(["Target"]).first()
grouped_and_first_df["N"] = subset_df.groupby(["Target"]).nunique()
grouped_and_first_df

new_df = grouped_and_first_df.reset_index()
new_df.columns = ["Name","first_date","N"]
new_df.tail(5)

,Name,first_date,N
13,C/2021 A6,21/01/2021,3
14,C/2021 A7,21/01/2021,6
15,C/2021 B3,13/02/2021,4
16,P/2020 V3,07/12/2020,3
17,P/2020 V4,10/12/2020,1


In [4]:
### 
target_location = '474' # Mt. John Observatory MPC code

obsDate_lst = []
for i in new_df["first_date"]:
    obsDate_lst.append(i)

targetname_lst = []
for i in new_df.Name:
    targetname_lst.append(i)

# need to convert epoch date to julian date
jd_obsDate_lst = []
for i in obsDate_lst:
    dt = dateutil.parser.parse(obsDate_lst[0]) 
    time = astropy.time.Time(dt)
    jd_obsDate_lst.append(time.jd)

In [5]:
# #need to query information from JPL Horizons database
eph_lst = []
el_lst = []
obj_lst = []
for index,jdDate in enumerate(jd_obsDate_lst):
    if targetname_lst[index] == "C/2018 F4":
        obj = Horizons(id='90004395', epochs=jdDate)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)
    else:
        obj = Horizons(id=targetname_lst[index], epochs=jdDate)
        obj_lst.append(obj)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)


# to check elements returned by obj.elements, see:
# https://astroquery.readthedocs.io/en/latest/api/astroquery.jplhorizons.HorizonsClass.html#astroquery.jplhorizons.HorizonsClass.elements

In [6]:
final_array = []
for index,target in enumerate(eph_lst):
    final_array.append([el_lst[index]['a'][0],eph_lst[index]['r'][0],
                        el_lst[index]['q'][0],el_lst[index]['Tp_jd'][0]])

In [7]:
eph_el_df = pd.DataFrame(final_array,columns=['a','r','q','T'])
eph_el_df.tail(5)

,a,r,q,T
13,-3.115937e-06,7.954310,4.928464,2.459550e+06
14,-3.258038e-06,2.855258,1.965796,2.459343e+06
15,-8.568433e-07,2.229828,1.878752,2.459258e+06
16,-6.356858e-06,6.230639,2.926078,2.458839e+06
17,-7.393996e-06,5.226928,1.507296,2.458846e+06


In [8]:
final_new_df = pd.concat([new_df,eph_el_df],axis=1)
final_new_df.tail(5)

,Name,first_date,N,a,r,q,T
13,C/2021 A6,21/01/2021,3,-3.115937e-06,7.954310,4.928464,2.459550e+06
14,C/2021 A7,21/01/2021,6,-3.258038e-06,2.855258,1.965796,2.459343e+06
15,C/2021 B3,13/02/2021,4,-8.568433e-07,2.229828,1.878752,2.459258e+06
16,P/2020 V3,07/12/2020,3,-6.356858e-06,6.230639,2.926078,2.458839e+06
17,P/2020 V4,10/12/2020,1,-7.393996e-06,5.226928,1.507296,2.458846e+06


In [9]:
final_new_df = final_new_df[['Name','first_date','a','N','r','q','T']]
final_new_df.tail(5)

,Name,first_date,a,N,r,q,T
13,C/2021 A6,21/01/2021,-3.115937e-06,3,7.954310,4.928464,2.459550e+06
14,C/2021 A7,21/01/2021,-3.258038e-06,6,2.855258,1.965796,2.459343e+06
15,C/2021 B3,13/02/2021,-8.568433e-07,4,2.229828,1.878752,2.459258e+06
16,P/2020 V3,07/12/2020,-6.356858e-06,3,6.230639,2.926078,2.458839e+06
17,P/2020 V4,10/12/2020,-7.393996e-06,1,5.226928,1.507296,2.458846e+06


In [10]:
final_new_df['T'] = final_new_df['T'].map(lambda name: Time(name, format='jd').to_value('iso')[:10])
final_new_df.head(5)

,Name,first_date,a,N,r,q,T
0,189040,21/01/2021,-0.000013,1,1.023791,0.091482,2021-01-26
1,2009 CD,11/02/2021,-0.000016,3,1.104592,0.102607,2021-01-30
2,2020 UB5,11/02/2021,-0.000490,4,0.983340,0.102455,2020-11-26
3,2021 CA1,11/02/2021,-0.000141,3,1.022337,0.011291,2021-02-08
4,2021 CD1,12/02/2021,-0.000010,3,1.139992,0.051302,2021-02-14


In [11]:
final_new_df['a'] = np.round(final_new_df['a'], decimals = 6)
final_new_df['r'] = np.round(final_new_df['r'], decimals = 2)
final_new_df['q'] = np.round(final_new_df['q'], decimals = 2)
final_new_df.head(2)

,Name,first_date,a,N,r,q,T
0,189040,21/01/2021,-0.000013,1,1.02,0.09,2021-01-26
1,2009 CD,11/02/2021,-0.000016,3,1.10,0.10,2021-01-30


In [12]:
final_new_df = final_new_df.drop(columns="first_date")

final_column_names = ['Name','$a_0$ [au]','N','$r_0$ [au]','$q$ [au]','T']
final_new_df.columns = final_column_names
final_new_df.head(2)

,Name,$a_0$ [au],N,$r_0$ [au],$q$ [au],T
0,189040,-0.000013,1,1.02,0.09,2021-01-26
1,2009 CD,-0.000016,3,1.10,0.10,2021-01-30


In [13]:
final_new_df.to_csv("final_look_table.csv")
final_new_df.to_latex(columns=final_column_names)

'\\begin{tabular}{llrrrrl}\n\\toprule\n{} &       Name &  \\$a\\_0\\$ [au] &  N &  \\$r\\_0\\$ [au] &  \\$q\\$ [au] &           T \\\\\n\\midrule\n0  &     189040 &   -0.000013 &  1 &        1.02 &      0.09 &  2021-01-26 \\\\\n1  &    2009 CD &   -0.000016 &  3 &        1.10 &      0.10 &  2021-01-30 \\\\\n2  &   2020 UB5 &   -0.000490 &  4 &        0.98 &      0.10 &  2020-11-26 \\\\\n3  &   2021 CA1 &   -0.000141 &  3 &        1.02 &      0.01 &  2021-02-08 \\\\\n4  &   2021 CD1 &   -0.000010 &  3 &        1.14 &      0.05 &  2021-02-14 \\\\\n5  &    2021 CH &   -0.000035 &  4 &        1.08 &      0.02 &  2021-02-10 \\\\\n6  &    2021 CM &   -0.000058 &  5 &        1.02 &      0.03 &  2021-02-06 \\\\\n7  &    2021 CP &   -0.000033 &  3 &        0.99 &      0.02 &  2021-02-03 \\\\\n8  &    2021 CS &   -0.000015 &  2 &        1.03 &      0.08 &  2021-02-01 \\\\\n9  &     360502 &   -0.000032 &  1 &        1.05 &      0.22 &  2021-01-16 \\\\\n10 &  C/2018 F4 &   -0.000002 &  3 &       

In [14]:
#WRITING TABLES!!!
final_look_table = Table.from_pandas(final_new_df)
ascii.write(final_look_table,format="latex",
            formats={'$a_0$ [au]':'%12.6f'})

\begin{table}
\begin{tabular}{cccccc}
Name & $a_0$ [au] & N & $r_0$ [au] & $q$ [au] & T \\
189040 & -0.000013 & 1 & 1.02 & 0.09 & 2021-01-26 \\
2009 CD & -0.000016 & 3 & 1.1 & 0.1 & 2021-01-30 \\
2020 UB5 & -0.000490 & 4 & 0.98 & 0.1 & 2020-11-26 \\
2021 CA1 & -0.000141 & 3 & 1.02 & 0.01 & 2021-02-08 \\
2021 CD1 & -0.000010 & 3 & 1.14 & 0.05 & 2021-02-14 \\
2021 CH & -0.000035 & 4 & 1.08 & 0.02 & 2021-02-10 \\
2021 CM & -0.000058 & 5 & 1.02 & 0.03 & 2021-02-06 \\
2021 CP & -0.000033 & 3 & 0.99 & 0.02 & 2021-02-03 \\
2021 CS & -0.000015 & 2 & 1.03 & 0.08 & 2021-02-01 \\
360502 & -0.000032 & 1 & 1.05 & 0.22 & 2021-01-16 \\
C/2018 F4 & -0.000002 & 3 & 10.85 & 10.37 & 2020-12-26 \\
C/2020 R7 & -0.000005 & 3 & 6.24 & 6.22 & 2020-11-02 \\
C/2021 A2 & -0.000001 & 1 & 1.41 & 0.5 & 2021-02-04 \\
C/2021 A6 & -0.000003 & 3 & 7.95 & 4.93 & 2021-12-01 \\
C/2021 A7 & -0.000003 & 6 & 2.86 & 1.97 & 2021-05-08 \\
C/2021 B3 & -0.000001 & 4 & 2.23 & 1.88 & 2021-02-12 \\
P/2020 V3 & -0.000006 & 3 & 6.23 &